## Agents Env


### Agent Workspace Environment

Each agent work in ubuntu environment:

```dockerfile
FROM ubuntu:latest

RUN apt-get update

RUN apt install git -y

RUN apt install python3 -y

RUN apt install python3-pip -y

RUN mkdir -p /agent_workspace

WORKDIR /agent_workspace

CMD ["/bin/bash"]
```


Create docker image:

```bash
docker build -t agent_env .
```


Basic prompt for agents:

````python

prompt = """Act as {role}. Your responsibility is {responsibility}.

You have access to an Ubuntu environment, previously updated and located in your work folder.```
````


### Create Agent Env


In [92]:
import docker
import time

client = docker.from_env()
container_ref = None
CONTAINER_IMAGE = "agent_env"


def create_container(container_name):
    global container_ref

    existing_containers = client.containers.list(
        all=True, filters={"name": CONTAINER_NAME}
    )
    if existing_containers:
        container = existing_containers[0]
        if container.status != "running":
            container.start()
            time.sleep(20)
        container.reload()
        container_ref = container
        return "Container criado com sucesso."

    container = client.containers.run(
        CONTAINER_IMAGE,
        name=container_name,
        detach=True,
        tty=True,
        stdin_open=True,
        working_dir="/agent_workspace",
        command="/bin/bash",
    )

    time.sleep(20)

    container_ref = container
    return "Container criado com sucesso."


CONTAINER_NAME = "qa_agent_container"
create_container(CONTAINER_NAME)

'Container criado com sucesso.'

In [74]:
def delete_container(container_name):
    existing_containers = client.containers.list(
        all=True, filters={"name": container_name}
    )

    if existing_containers:
        container = existing_containers[0]
        container.stop()
        container.remove()
        return "Container deletado com sucesso."


# CONTAINER_NAME = "qa_agent_container"
# delete_container(CONTAINER_NAME)

In [93]:
def get_container():
    global container_ref

    if container_ref:
        container_ref.reload()
        if container_ref.status == "running":
            return container_ref

    containers = client.containers.list(all=True, filters={"name": CONTAINER_NAME})
    if containers:
        container = containers[0]
        if container.status != "running":
            container.start()
            time.sleep(60)
        container.reload()
        container_ref = container
        return container

    return None

### Utils function (get_container)


In [104]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama

In [105]:
# model = "llama3:8b"
model_name = "mistral-nemo"

# llm = OllamaLLM(model=model_name)
model = ChatOllama(model=model_name)

### Run Command Tool (basic tool):


In [106]:
@tool
def terminal(command: str) -> str:
    """
    Execute a shell command to be executed.
    """

    container = get_container()
    if not container:


        return "No container found. Please try again."


    try:

        result = container.exec_run(f"/bin/sh -c '{command}'")

        return result.output.decode("utf-8")

    except Exception as e:

        return f"Error executing command: {str(e)}"

In [109]:
# Test run_command (about mkdir)
print(terminal("mkdir --version"))

# Test run_command (create directory)
print(terminal("mkdir test"))

# Test run_command (ls)
print(terminal("ls"))

# Test run_command (cd test && ls)
print(terminal("cd test && ls"))

# Test run_command (rm -rf test)
print(terminal("rm -rf test"))

# Test run_command (ls)
print(terminal("ls"))

mkdir (GNU coreutils) 9.4
Copyright (C) 2023 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

Written by David MacKenzie.


test






In [110]:
tools = [terminal]
memory = MemorySaver()
agent_executor = create_react_agent(
    name="qa_agent",
    tools=tools,
    model=model,
)

description = """You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  
Follow the instructions carefully and execute the tasks precisely.  

System Information:
- OS: Ubuntu (root access available).
- Shell: Bash (no interactive commands allowed)."""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", description),
        ("user", "Perform the following tasks:\n\n{tasks}"),
    ]
)


tasks = """Steps to follow:
1. Create the 'projects' folder in the current directory.
2. Access the 'projects' folder and clone the repository: https://github.com/astropy/astropy
3. Access the 'projects/astropy' and checkout the specific commit: d16bfe05a744909de4b27f5875fe0d4ed41ce607."""

prompt = prompt_template.invoke({"tasks": tasks})

response = agent_executor.invoke(prompt, {"recursion_limit": 100})
print(response)

{'messages': [SystemMessage(content='You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  \nFollow the instructions carefully and execute the tasks precisely.  \n\nSystem Information:\n- OS: Ubuntu (root access available).\n- Shell: Bash (no interactive commands allowed).', additional_kwargs={}, response_metadata={}, id='5a865215-7008-47b0-9b53-fc13cdffa555'), HumanMessage(content="Perform the following tasks:\n\nSteps to follow:\n1. Create the 'projects' folder in the current directory.\n2. Access the 'projects' folder and clone the repository: https://github.com/astropy/astropy\n3. Access the 'projects/astropy' and checkout the specific commit: d16bfe05a744909de4b27f5875fe0d4ed41ce607.", additional_kwargs={}, response_metadata={}, id='eef8831d-068e-4f21-af41-9d35bdb47945'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral-nemo', 'created_at': '2025-02-19T17:32:00.5693476Z', 'done': True, 'done_reaso

In [111]:
tasks = """Steps to follow:
1. Access the 'projects/astropy' folder and install the requirements.
2. Run the tests."""

prompt = prompt_template.invoke({"tasks": tasks})

response = agent_executor.invoke(prompt, {"recursion_limit": 100})
print(response)

{'messages': [SystemMessage(content='You are a Software Quality Analyst (QA) responsible for setting up and preparing the project environment.  \nFollow the instructions carefully and execute the tasks precisely.  \n\nSystem Information:\n- OS: Ubuntu (root access available).\n- Shell: Bash (no interactive commands allowed).', additional_kwargs={}, response_metadata={}, id='795f215c-872e-4779-b343-3bb859c6c0bb'), HumanMessage(content="Perform the following tasks:\n\nSteps to follow:\n1. Access the 'projects/astropy' folder and install the requirements.\n2. Run the tests.", additional_kwargs={}, response_metadata={}, id='5d3ef649-834d-4ffb-9af6-bafaa9bc3d98'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral-nemo', 'created_at': '2025-02-19T17:32:55.4414162Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2606449700, 'load_duration': 22721100, 'prompt_eval_count': 142, 'prompt_eval_duration': 492000000, 'eval_count': 30, 'eval_duration': 20880000